In [13]:
import cv2
import itertools
import numpy as np
import mediapipe as mp
import pandas as pd

In [14]:
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(
    refine_landmarks=True,
    static_image_mode=True,
    max_num_faces = 1,
)

mp_drawing = mp.solutions.drawing_utils
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)

for i in range(100):
    image = cv2.imread("./dataset/train/img/%s.png"%i)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    globals()['height'+str(i)] = image.shape[0]
    
    results = face_mesh.process(image)
    
    for single_face_landmarks in results.multi_face_landmarks:
        mp_drawing.draw_landmarks(
            image=image,
            landmark_list = single_face_landmarks,
            connections=mp_face_mesh.FACEMESH_CONTOURS,
            landmark_drawing_spec = drawing_spec,            
        )
    
    INDEX_LIST = [127, 234, 93, 132, 58, 172, 136, 150, 149, 
             176, 148, 152, 377, 400, 378, 379, 365, 397, 288, 361, 323, 454, 356]
    
    globals()['facemesh'+str(i)] = np.zeros((len(INDEX_LIST), 2))
    
    for single_face_landmarks in results.multi_face_landmarks:
        for j, k in enumerate(INDEX_LIST):
            coordinates = single_face_landmarks.landmark[k]
            globals()['facemesh'+str(i)][j][0] = coordinates.x * image.shape[1]
            globals()['facemesh'+str(i)][j][1] = coordinates.y * image.shape[0]

In [15]:
for i in range(100):
    land = np.zeros((len(facemesh0),2))
    
    for j in range(len(facemesh0)):
        land[j][0] = globals()['facemesh'+str(i)][j][0]
        land[j][1] = globals()['facemesh'+str(i)][j][1]
        
    x = land[:, 0]
    y = land[:, 1] * (-1) + globals()['height'+str(i)]
    
    x -= ((min(x) + max(x)) / 2)
    y -= ((min(y) + max(y)) / 2)
    
    norm_x = (x - min(x)) / (max(y)-min(y))
    norm_y = (y - min(y)) / (max(y)-min(y))
    
    globals()['norm'+str(i)] = np.column_stack((norm_x, norm_y))
    
    globals()['norm'+str(i)] = globals()['norm'+str(i)].flatten()

In [16]:
train_x = np.zeros((100, len(norm0)))
for i in range(100):
    for j in range(len(norm0)):
        train_x[i][j] = globals()['norm'+str(i)][j]

In [17]:
for i in range(100):
    image = cv2.imread("./dataset/test/img/%s.png"%i)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    globals()['height'+str(i)] = image.shape[0]
    
    results = face_mesh.process(image)
    
    for single_face_landmark in results.multi_face_landmarks:
        mp_drawing.draw_landmarks(
            image=image,
            landmark_list = single_face_landmarks,
            connections=mp_face_mesh.FACEMESH_CONTOURS,
            landmark_drawing_spec = drawing_spec,            
        )
    
    INDEX_LIST = [127, 234, 93, 132, 58, 172, 136, 150, 149, 
             176, 148, 152, 377, 400, 378, 379, 365, 397, 288, 361, 323, 454, 356]
    
    globals()['facemesh'+str(i)] = np.zeros((len(INDEX_LIST), 2))
    
    for single_face_landmarks in results.multi_face_landmarks:
        for j, k in enumerate(INDEX_LIST):
            coordinates = single_face_landmarks.landmark[k]
            globals()['facemesh'+str(i)][j][0] = coordinates.x * image.shape[1]
            globals()['facemesh'+str(i)][j][1] = coordinates.y * image.shape[0]

In [18]:
for i in range(100):
    land = np.zeros((len(facemesh0),2))
    
    for j in range(len(facemesh0)):
        land[j][0] = globals()['facemesh'+str(i)][j][0]
        land[j][1] = globals()['facemesh'+str(i)][j][1]
        
    x = land[:, 0]
    y = land[:, 1] * (-1) + globals()['height'+str(i)]
    
    x -= ((min(x) + max(x)) / 2)
    y -= ((min(y) + max(y)) / 2)
    
    norm_x = (x - min(x)) / (max(y)-min(y))
    norm_y = (y - min(y)) / (max(y)-min(y))
    
    globals()['norm'+str(i)] = np.column_stack((norm_x, norm_y))
    
    globals()['norm'+str(i)] = globals()['norm'+str(i)].flatten()

In [19]:
test_x = np.zeros((100, len(norm0)))
for i in range(100):
    for j in range(len(norm0)):
        test_x[i][j] = globals()['norm'+str(i)][j]

In [20]:
train_y = pd.read_csv('./dataset/train/train.csv')
train_y = train_y.iloc[:, 5:9]
test_y = pd.read_csv('./dataset/test/test.csv')
test_y = test_y.iloc[:, 5:9]

In [21]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

model = LinearRegression()
model.fit(train_x, train_y)

LinearRegression()

In [22]:
y_pred = model.predict(test_x)
mae = mean_absolute_error(test_y, y_pred)
mae

0.20284023535884432

In [23]:
pred = pd.DataFrame(y_pred)
pred.columns = ['jawsPosition','jawsSize','chinPosition','chinSize']
pred

,jawsPosition,jawsSize,chinPosition,chinSize
0,0.661680,0.396611,0.596810,0.320424
1,-0.068711,0.791984,0.241495,0.691124
2,0.576094,0.739738,0.667230,0.577759
3,0.304572,0.708258,0.481912,0.572240
4,0.371704,0.269348,0.581494,0.577859
...,...,...,...,...
95,0.836781,0.582406,0.868739,0.076085
96,0.289923,0.743007,0.533207,0.403779
97,0.067863,0.509438,-0.373487,0.625928
98,0.699921,0.555043,0.706328,0.308237


In [24]:
test_y

,jawsPosition,jawsSize,chinPosition,chinSize
0,0.323832,0.595424,0.420894,0.251898
1,0.252126,0.766253,0.768240,0.637456
2,0.466237,0.531876,0.502866,0.621738
3,0.289253,0.519737,0.526797,0.793071
4,0.225629,0.502349,0.339251,0.652542
...,...,...,...,...
95,0.320538,0.766127,0.231935,0.399568
96,0.530781,0.438227,0.651389,0.252586
97,0.565084,0.416299,0.286769,0.596652
98,0.569411,0.301160,0.784193,0.403280
